In [65]:
using SymPy

function matriz_S(x) #função para gerar matriz de estratégias
    S = vec(collect(Base.product(reverse(x)...))) #produto cartesiano
    #transformar conjunto de estratégias em matriz
    S_M = []
    for i = 1:length(S[1])
        d = [y[i] for y in S]
        if i == 1 S_M = [d; ] else S_M = [S_M d] end
    end
    S_M = reverse(S_M, dims = 2)
    return S_M
end


function melhores_respostas(x,y) #S e U
ncol = size(x, 2) #numero de jogadores
BR = copy(x) #matriz contendo as melhores respostas
for i = 1:ncol #para cada agente i
    S_not_i = sum(x[:, 1:end .!= i], dims=2) # possiveis estratégias dos oponentes de i
    for j in unique(S_not_i) 
        k = findall(S_not_i .== j) #conjunto de estratégias contendo a estratégia j
        l = first.(Tuple.(k)) # linhas referentes à estratégia j
        h = findall(y[l,i] .== max(maximum(y[l,i]))) #estratégias do indivíduo i que maximisam sua utilidade frente à estrategia J
        #criar matriz BR
        for ll in l[h] BR[ll,i] = Sym(string(BR[ll,i],"^*")) end #assinalar melhores respostas com *
    end
end
return BR
end

function EN_Jogo_sequencial(S,U)
BR = copy(S)
i = 0
for t = reverse(1:size(S, 2)) #para cada período t
    i = i + 1 #jogador i
    r = []
    S_not_i = sum(S[:, 1:(t-1)], dims=2) #estratégias dos oponentes 
    for j in unique(S_not_i) 
        ll = findall(S_not_i .== j) #linhas contendo a estratégia j
        l = first.(Tuple.(ll))
        br = findall(U[l,i] .== (maximum(U[l,i]))) #encontrar BR
        for ll in l[br] BR[ll,i] = Sym(string(BR[ll,i],"^*")) end
    end
end
BR
end



EN_Jogo_sequencial (generic function with 1 method)

### Exemolo 1

In [7]:
a1 = [Sym("a_$i^$j") for i=1 for j=["b","t"]] #acoes do agente 1
a2 = [Sym("a_$i^$j") for i=2 for j=["d","u"]] #acoes do agente 2
A = [a1 , a2]
S = matriz_S(A)

U = [4 2;1 1;5 1;2 2]


4×2 Array{Int64,2}:
 4  2
 1  1
 5  1
 2  2

In [58]:
melhores_respostas(S,U)

4×2 Array{Sym,2}:
   a_1__b  a_2__d__*
   a_1__b    a_2__u
 a_1__t__*    a_2__d
 a_1__t__*  a_2__u__*

In [57]:
EN_Jogo_sequencial(S,U)

4×2 Array{Sym,2}:
 a_1__b__*  a_2__d__*
   a_1__b    a_2__u
 a_1__t__*    a_2__d
   a_1__t  a_2__u__*

### Exemplo 2

In [70]:
a1_1t = [Sym("a_$i^$j") for i=1 for j=1:2] #acoes do agente 1 no tempo 1
a2_2t = [Sym("a_$i^$j") for i=2 for j=1:2] #acoes do agente 2 no tempo 2
a1_3t = [Sym("a_$i^$j") for i=1 for j=1:2] #acoes do agente 1 no tempo 3
A = [a1_1t , a2_2t , a1_3t] # conjunto de todas as ações
S = matriz_S(A) 

#matriz utilidade
U = [2 2 2;3 1 3;2 0 2;3 2 3;0 5 0;1 5 1;2 2 2;3 2 3]

en = EN_Jogo_sequencial(S,U)

8×3 Array{Sym,2}:
   a¹₁  a_2__1__*    a¹₁
 a_1__1__*    a¹₂  a_1__2__*
   a¹₁    a²₂    a¹₁
 a_1__1__*  a_2__2__*  a_1__2__*
   a²₁  a_2__1__*    a¹₁
 a_1__2__*  a_2__1__*    a²₁
   a²₁    a²₂    a¹₁
 a_1__2__*    a²₂  a_1__2__*

In [67]:
using Latexify
print(latexify(en))

\begin{equation}
\left[
\begin{array}{ccc}
a_1.1^1 & a_2.2^1^* & a_1.3^1 \\
a_1.1^1^* & a_2.2^1 & a_1.3^2^* \\
a_1.1^1 & a_2.2^2 & a_1.3^1 \\
a_1.1^1^* & a_2.2^2^* & a_1.3^2^* \\
a_1.1^2 & a_2.2^1^* & a_1.3^1 \\
a_1.1^2^* & a_2.2^1^* & a_1.3^2 \\
a_1.1^2 & a_2.2^2 & a_1.3^1 \\
a_1.1^2^* & a_2.2^2 & a_1.3^2^* \\
\end{array}
\right]
\end{equation}
